In [30]:
import os 
import pymysql
import re
from langgraph.graph import END
from typing import List,TypedDict, Dict, Any, Optional
from mistralai import Mistral
from dotenv import load_dotenv
from langchain.chat_models import init_chat_model
from langchain_mistralai import MistralAIEmbeddings
from langchain_core.vectorstores import InMemoryVectorStore
from langchain.document_loaders import DirectoryLoader,TextLoader
from langchain import hub
from langgraph.graph import START, StateGraph
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
import logging
#logging.basicConfig(level=logging.DEBUG)






In [31]:

load_dotenv()

tracing = os.getenv("LANGSMITH_TRACING")
api_key = os.getenv("LANGSMITH_API_KEY")
Mistral_api_key = Mistral(api_key=os.getenv("MISTRAL_API_KEY"))

# Chemin vers le répertoire contenant les bases de connaissances
DATA_ROOT = "../data"
ECHANGES_PATH = os.path.join(DATA_ROOT, "echanges")
REGLES_PATH = os.path.join(DATA_ROOT, "regles") 
OFFICIAL_DOCS_PATH = os.path.join(DATA_ROOT,"docs_officiels")

# Chat model mistral
llm = init_chat_model("mistral-large-latest", model_provider="mistralai")


DEBUG:asyncio:Using selector: SelectSelector


In [32]:
class DatabaseManager:
    def __init__(self):
        load_dotenv()
        self.config = {
            'user': os.getenv('SQL_USER'),
            'password': os.getenv('SQL_PASSWORD', ''),
            'host': os.getenv('SQL_HOST', 'localhost'),
            'database': os.getenv('SQL_DB'),
            'port': int(os.getenv('SQL_PORT', '3306'))
        }
        if not all([self.config['user'], self.config['host'], self.config['database']]):
            raise ValueError("Variables essentielles manquantes : SQL_USER, SQL_HOST ou SQL_DB.")
        
    
    def tester_connexion(self) -> bool:
        try:
            conn = pymysql.connect(**self.config)
            curseur = conn.cursor()
            curseur.execute("SELECT 1")
            curseur.fetchone()
            curseur.close()
            conn.close()
            print(" Connexion réussie avec pymysql.")
            return True
        except pymysql.Error as erreur:
            print(f" Échec de la connexion : {erreur}")
            return False
    
    def rechercher_dossier(self, numero_dossier: Optional[str] = None, **kwargs) -> List[Dict[str, Any]]:
        try:
            conn = pymysql.connect(**self.config)
            curseur = conn.cursor(pymysql.cursors.DictCursor)  # Pour des résultats sous forme de dictionnaires
            conditions = []
            parametres = []

            if numero_dossier:
                conditions.append("Numero = %s")
                parametres.append(numero_dossier)

            for cle, valeur in kwargs.items():
                if valeur is not None:
                    conditions.append(f"{cle} = %s")
                    parametres.append(valeur)

            requete = f"SELECT * FROM dossiers WHERE {' AND '.join(conditions) if conditions else '1=1'}"
            curseur.execute(requete, parametres)
            resultats = curseur.fetchall()
            curseur.close()
            conn.close()
            return resultats
        except pymysql.Error as erreur:
            print(f" Erreur de recherche : {erreur}")
            return []
"""  
if __name__ == "__main__":
    try:
        db_manager = DatabaseManager()
        if db_manager.tester_connexion():
            resultats = db_manager.rechercher_dossier(numero_dossier='82-2069')
            print("\n📋 Résultats de la recherche :")
            for dossier in resultats:
                print(dossier)
        else:
            print(" Échec de la connexion.")
    except Exception as e:
        print(f" Erreur critique : {e}")
        traceback.print_exc()
"""

'  \nif __name__ == "__main__":\n    try:\n        db_manager = DatabaseManager()\n        if db_manager.tester_connexion():\n            resultats = db_manager.rechercher_dossier(numero_dossier=\'82-2069\')\n            print("\n📋 Résultats de la recherche :")\n            for dossier in resultats:\n                print(dossier)\n        else:\n            print(" Échec de la connexion.")\n    except Exception as e:\n        print(f" Erreur critique : {e}")\n        traceback.print_exc()\n'

In [33]:
# Fonction pour extraire le numéro de dossier de la question
def extract_dossier_number(question: str) -> List[str]:
    # Pattern pour rechercher les numéros de dossier (format XX-XXXX ou similaire)
    patterns = [
        r'\b\d{2}-\d{4}\b',  # Format 82-2069
        r'\bdossier\s+(?:n°|numéro|numero|n|°|)?\s*(\w+-\w+)\b',  # Format avec "dossier n° XXX-XXX"
        r'\bdossier\s+(?:n°|numéro|numero|n|°|)?\s*(\d+)\b'  # Format avec "dossier n° XXX"
    ]
    
    results = []
    for pattern in patterns:
        matches = re.findall(pattern, question, re.IGNORECASE)
        if matches:
            results.extend(matches)
    
    return list(set(results))  # Éliminer les doublons
    
   

# Fonction pour rechercher les informations d'un dossier dans la base de données via DatabaseManager
def search_dossier_in_db(dossier_numbers: List[str]) -> List[Dict[str, Any]]:
    try:
        db_manager = DatabaseManager()
        
        # Tester la connexion avant de procéder
        if not db_manager.tester_connexion():
            print("Impossible de se connecter à la base de données")
            return []
        
        results = []
        for num in dossier_numbers:
            dossier_results = db_manager.rechercher_dossier(numero_dossier=num)
            results.extend(dossier_results)
        
        return results
    except Exception as e:
        print(f"Erreur lors de la recherche dans la base de données: {e}")
        return []

        
def db_resultats_to_documents(resultats: List[Dict[str,Any]]) -> List[Document]:
    documents = [] 
    for resultat in resultats:
        #contenu formaté à partir des donne du dossier
        content = f"""
        - Informations sur le dossier {resultat.get('Numero', 'N/A')}:
        - Nom de l'usager: {resultat.get('nom_usager', 'N/A')}
        - Date de création: {resultat.get('date_creation', 'N/A')}
        - Dernière modification: {resultat.get('derniere_modification', 'N/A')}
        - Agent affecté: {resultat.get('agent_affecter', 'N/A')}
        - Instructeur: {resultat.get('instructeur', 'N/A')}
        - Statut actuel: {resultat.get('statut', 'N/A')}
        - Statut visible par l'usager: {resultat.get('statut_visible_usager', 'N/A')}
        - Montant: {resultat.get('montant', 'N/A')} €
        """
        
        #création d'un document Langchain avec le meta data
        doc = Document(
            page_content=content,
            metadata={
                "source": "base_de_donnees",
                "type":"dossier",
                "numéro": resultat.get('Numero','N/A'),
                "section":"Informations dossier",
                "page": "1", 
                "update_date": resultat.get('derniere_modification', 'N/A')
                
            }
        )
        
        documents.append(doc)
        
    return documents

In [34]:
#Fonction pour charger tous les documents des différents dossiers
def load_all_documents():
    all_docs = []
    
    # On va d'abord charger les documents des echanges 
    try:
        echanges_loader = DirectoryLoader(
            ECHANGES_PATH,
            glob="**/*txt",  # Charger tous les .txt, y compris dans les sous-dossiers
            loader_cls=TextLoader,
            loader_kwargs={"encoding": "utf-8"},
            recursive=True  # Charger récursivement dans les sous-dossiers
        )
        echanges_docs = echanges_loader.load()
        for doc in echanges_docs:
            doc.metadata["category"] = "echanges"
        print(f"{len(echanges_docs)} documents d'echanges chargés.")
        all_docs.extend(echanges_docs)
    except Exception as e:
        print(f"Erreur lors du chargement des dossiers d'echanges:{e}")
        
    #Ici on va charger les documents des regles
    try:
        regles_loader = DirectoryLoader(
            REGLES_PATH,
            glob="*.txt",
            loader_cls=TextLoader,
            loader_kwargs={"encoding": "utf-8"}
        )
        
        regles_docs = regles_loader.load()
        for doc in regles_docs:
            doc.metadata["category"] = "regles"
        print(f"{len(regles_docs)} documents de regles chargés.")
        all_docs.extend(regles_docs)
    except Exception as e:
        print(f"Erreur lors du changement des documents de regles: {e}")
        
    #Ici on va charger les documents des docs_officiels
    try:
        official_docs_loader = DirectoryLoader(
            OFFICIAL_DOCS_PATH,
            glob="*.txt",
            loader_cls=TextLoader,
            loader_kwargs={"encoding": "utf-8"}
        ) 
        official_docs = official_docs_loader.load()
        for doc in official_docs:
            doc.metadata["category"] = "docs_officiels"
        print(f"{len(official_docs)} documents de docs_officiels chargés.")
        all_docs.extend(official_docs)
    except Exception as e:
        print(f"Erreur lors du chargement des documetns officiels: {e}")
    
    print(f"Total : {len(all_docs)} documents chargés dans la base de connaissances")
    return all_docs
            
docs = load_all_documents()

DEBUG:langchain_community.document_loaders.directory:Processing file: ..\data\echanges\echange1.txt
DEBUG:langchain_community.document_loaders.directory:Processing file: ..\data\echanges\echange10.txt
DEBUG:langchain_community.document_loaders.directory:Processing file: ..\data\echanges\echange11.txt
DEBUG:langchain_community.document_loaders.directory:Processing file: ..\data\echanges\echange12.txt
DEBUG:langchain_community.document_loaders.directory:Processing file: ..\data\echanges\echange13.txt
DEBUG:langchain_community.document_loaders.directory:Processing file: ..\data\echanges\echange14.txt
DEBUG:langchain_community.document_loaders.directory:Processing file: ..\data\echanges\echange15.txt
DEBUG:langchain_community.document_loaders.directory:Processing file: ..\data\echanges\echange16.txt


DEBUG:langchain_community.document_loaders.directory:Processing file: ..\data\echanges\echange17.txt
DEBUG:langchain_community.document_loaders.directory:Processing file: ..\data\echanges\echange18.txt
DEBUG:langchain_community.document_loaders.directory:Processing file: ..\data\echanges\echange19.txt
DEBUG:langchain_community.document_loaders.directory:Processing file: ..\data\echanges\echange2.txt
DEBUG:langchain_community.document_loaders.directory:Processing file: ..\data\echanges\echange20.txt
DEBUG:langchain_community.document_loaders.directory:Processing file: ..\data\echanges\echange21.txt
DEBUG:langchain_community.document_loaders.directory:Processing file: ..\data\echanges\echange22.txt
DEBUG:langchain_community.document_loaders.directory:Processing file: ..\data\echanges\echange23.txt
DEBUG:langchain_community.document_loaders.directory:Processing file: ..\data\echanges\echange24.txt
DEBUG:langchain_community.document_loaders.directory:Processing file: ..\data\echanges\echan

46 documents d'echanges chargés.
2 documents de regles chargés.
2 documents de docs_officiels chargés.
Total : 50 documents chargés dans la base de connaissances


In [35]:
# Découper les documents en chunks pour un meilleur traitement par le modèle
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
all_splits = text_splitter.split_documents(docs)

In [36]:
# Initialiser les embeddings Mistral
embeddings = MistralAIEmbeddings()

DEBUG:urllib3.connectionpool:Resetting dropped connection: huggingface.co
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /mistralai/Mixtral-8x7B-v0.1/resolve/main/tokenizer.json HTTP/1.1" 401 0
c:\Python312\Lib\site-packages\langchain_mistralai\embeddings.py:181: UserWarning: Could not download mistral tokenizer from Huggingface for calculating batch sizes. Set a Huggingface token via the HF_TOKEN environment variable to download the real tokenizer. Falling back to a dummy tokenizer that uses `len()`.
  warnings.warn(


In [37]:
# Créer le vectorstore en mémoire à partir des chunks et des embeddings
vector_store = InMemoryVectorStore.from_documents(all_splits, embeddings)

DEBUG:httpcore.connection:connect_tcp.started host='api.mistral.ai' port=443 local_address=None timeout=120 socket_options=None
DEBUG:httpcore.connection:connect_tcp.complete return_value=<httpcore._backends.sync.SyncStream object at 0x0000023A045DA9F0>
DEBUG:httpcore.connection:start_tls.started ssl_context=<ssl.SSLContext object at 0x0000023A02D16550> server_hostname='api.mistral.ai' timeout=120
DEBUG:httpcore.connection:start_tls.complete return_value=<httpcore._backends.sync.SyncStream object at 0x0000023A047F78C0>
DEBUG:httpcore.http11:send_request_headers.started request=<Request [b'POST']>
DEBUG:httpcore.http11:send_request_headers.complete
DEBUG:httpcore.http11:send_request_body.started request=<Request [b'POST']>
DEBUG:httpcore.http11:send_request_body.complete
DEBUG:httpcore.http11:receive_response_headers.started request=<Request [b'POST']>
DEBUG:httpcore.http11:receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Date', b'Fri, 04 Apr 2025 10:29:06 GM

In [ ]:
# Définir le prompt pour la recherche de réponse
prompt = hub.pull("rlm/rag-prompt")

# Définition de l'état de l'application
class State(TypedDict):
    question: str
    context: List[Document]  # Liste d'objets Document
    db_results: List[Dict[str,Any]] # Résultats de la base de données
    answer: str

# Fonction de récupération des données de la base de données
def search_database(state: State) -> Dict[str, Any]:
    # Extraire les numéros de dossier de la question
    dossier_numbers = extract_dossier_number(state["question"])
    
    # Si des numéros de dossier sont trouvés, rechercher dans la base de données
    db_results = []
    if dossier_numbers:
        db_results = search_dossier_in_db(dossier_numbers)
    
    return {"db_results": db_results}
 
    
# Fonction de récupération (retrieval) des documents pertinents
def retrieve(state: State) -> Dict[str, Any]:
    try:
        # Récupérer les 3 documents les plus pertinents depuis le vectorstore
        retrieved_docs = vector_store.similarity_search(state["question"], k=5)
        
        # Convertir les résultats de la base de données en documents
        db_docs = db_resultats_to_documents(state["db_results"])
        
        # Combiner les documents de la base de données et les documents du vectorstore
        combined_docs = db_docs + retrieved_docs
        
        return {"context": combined_docs}
    except Exception as e:
        print(f"Erreur dans la fonction retrieve: {e}")
        return {"context": []}  # Retourner une liste vide en cas d'erreur


# Fonction de génération de la réponse
def generate(state: State) -> Dict[str, Any]:
    try:
        # Vérifier si le contexte est vide
        if not state["context"]:
            return {"answer": "Je n'ai pas trouvé d'informations pertinentes pour répondre à votre question."}
        
        # Agrégation du contenu des documents récupérés avec détails sur les sources
        docs_details = []
        for doc in state["context"]:  
            source = doc.metadata.get("source", "Source inconnue")
            category = doc.metadata.get("category", "non classifié")
            
            # Traitement spécial pour les documents de la base de données
            if source == "base_de_donnees":
                file_name = f"Base de données - Dossier {doc.metadata.get('numéro', 'inconnu')}"
                section = doc.metadata.get("section", "Section non spécifiée")
                page = "N/A"
                update_date = doc.metadata.get("update_date", "Date non disponible")
            else:
                file_name = os.path.basename(source)
                section = doc.metadata.get("section", "Section non spécifiée")
                page = doc.metadata.get("page", "Page non spécifiée")
                update_date = doc.metadata.get("update_date", "Date non disponible")

            docs_details.append({
                "content": doc.page_content,
                "file_name": file_name,
                "section": section,
                "page": page,
                "update_date": update_date,
                "category": category
            })

        # Agrégation du contenu des documents
        docs_content = "\n\n".join(doc["content"] for doc in docs_details)

        # Formatage des sources avec catégorie, section, page et date
        formatted_sources = "\n".join([
            f"[Document: {doc['file_name']}, Catégorie: {doc['category']}, Section: {doc['section']}, Page: {doc['page']}, Mise à jour: {doc['update_date']}]"
            for doc in docs_details
        ])
        
        # Instructions système mises à jour selon les nouvelles exigences
        system_instructions = (
            "Tu es un instructeur expert du dispositif KAP Numérique. Tu réponds à des questions en te basant uniquement sur les informations fournies dans le contexte.\n\n"
            
            "Consignes de réponse :\n"
            "1. Commence ta réponse en répétant la question posée, par exemple : 'En réponse à votre question : \"[question]\", voici les informations demandées :'\n"
            "2. Fournis une réponse concise et structurée.\n"
            "3. Utilise des phrases et des listes à puces pour organiser les informations.\n"
            
            "4. Traitement des questions sur un dossier spécifique :\n"
            "   - Vérifie d'abord les informations de la base de données.\n"
            "   - Indique clairement le statut actuel du dossier, la date de dernière modification, et les informations pertinentes du demandeur.\n"
            "   - Consulte ensuite les documents officiels et les règles pour expliquer les procédures.\n"
            "   - Examine les exemples d'échanges similaires pour adapter le style et le contenu de ta réponse.\n"
            
            
            "5. Traitement des questions générales sur le dispositif KAP Numérique :\n"
            "   - Consulte en priorité les documents officiels puis les règles.\n"
            "   - Utilise les exemples d'échanges pour adapter le format de ta réponse et son niveau de détail.\n"
            
            "6. Limitations :\n"
            "   - Si la question ne concerne ni le dispositif KAP Numérique, ni un bénéficiaire du programme, indique clairement que tu ne peux pas traiter ce type de demande.\n"
            "   - Exemple: 'Votre demande ne semble pas concerner le dispositif KAP Numérique ou l'un de ses bénéficiaires. Je ne peux malheureusement pas traiter ce type de requête.'\n"
            
            "7. Priorisation des sources :\n"
            "   - Documents officiels ('officiel') > Règles ('regles') > Échanges ('echanges')\n"
            "   - Les informations issues de la base de données sont prioritaires pour les questions sur un dossier spécifique.\n"
            
            "8. Cites systématiquement les sources avec le format suivant : [Document: Nom du document, Catégorie: Type de document, Section: Nom de la section, Page: Numéro de page, Mise à jour: Date].\n"
        )
        
        # Construction de l'invite utilisateur
        user_prompt = f"Question: {state['question']}\n\nContexte extrait des documents et de la base de données:\n{docs_content}"
        
        # Messages combinant instructions système et question de l'utilisateur
        messages = [
            {"role": "system", "content": system_instructions},
            {"role": "user", "content": user_prompt}
        ]
        
        # Appel au modèle LLM avec gestion des erreurs
        try:
            response = llm.invoke(messages)
            return {"answer": response.content}
        except Exception as e:
            return {"answer": f"Erreur lors de la génération de la réponse : {e}"}
    except Exception as e:
        print(f"Erreur dans la fonction generate: {e}")
        return{"answer":f"Une erreur s'est produite lors du traitement de votre demande: {e}"}
    
    
# Construction du graphe d'application
def build_graph():
    try:
        # Définir les nœuds du graphe
        workflow = StateGraph(State)
        
        # Ajouter les nœuds individuellement pour mieux contrôler
        workflow.add_node("search_database", search_database)
        workflow.add_node("retrieve", retrieve)
        workflow.add_node("generate", generate)
        
        # Définir les transitions entre les nœuds
        workflow.add_edge(START, "search_database")
        workflow.add_edge("search_database", "retrieve")
        workflow.add_edge("retrieve", "generate")
        workflow.add_edge("generate", END)
        
        # Compiler le graphe
        return workflow.compile()
    except Exception as e:
        print(f"Erreur lors de la construction du graphe: {e}")
        raise
        





DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.smith.langchain.com:443
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (2): api.smith.langchain.com:443
DEBUG:urllib3.connectionpool:https://api.smith.langchain.com:443 "GET /info HTTP/1.1" 200 672
DEBUG:urllib3.connectionpool:https://api.smith.langchain.com:443 "GET /commits/rlm/rag-prompt/latest HTTP/1.1" 200 844


In [39]:
# On initialise le graphe
graph = build_graph()

if __name__ == "__main__":
    print("Tapez 'exit' pour quitter.")
    
    # Tester d'abord la connexion à la base de données
    try:
        db_manager = DatabaseManager()
        if not db_manager.tester_connexion():
            print("Avertissement: Impossible de se connecter à la base de données. Les requêtes sur les dossiers ne fonctionneront pas.")
    except Exception as e:
        print(f"Erreur lors de l'initialisation de la connexion à la base de données: {e}")
    
    while True:
        try:
            user_query = input("\nPosez votre question : ")
            if user_query.lower() in ["exit", "quit"]:
                break
                
            # Initialiser l'état avec des listes vides pour context et db_results
            initial_state = {
                "question": user_query,
                "context": [],
                "db_results": [],
                "answer": ""
            }
            
            # Invoquer le graphe avec gestion d'erreur
            try:
                state = graph.invoke(initial_state)
                print("\nRéponse :", state["answer"])
            except Exception as e:
                print(f"\nErreur lors de l'exécution du graphe: {e}")
                
        except KeyboardInterrupt:
            print("\nProgramme interrompu par l'utilisateur.")
            break
        except Exception as e:
            print(f"\nErreur inattendue: {e}")

Tapez 'exit' pour quitter.
 Connexion réussie avec pymysql.
 Connexion réussie avec pymysql.


DEBUG:httpcore.connection:close.started
DEBUG:httpcore.connection:close.complete
DEBUG:httpcore.connection:connect_tcp.started host='api.mistral.ai' port=443 local_address=None timeout=120 socket_options=None
DEBUG:httpcore.connection:connect_tcp.complete return_value=<httpcore._backends.sync.SyncStream object at 0x0000023A0278E5A0>
DEBUG:httpcore.connection:start_tls.started ssl_context=<ssl.SSLContext object at 0x0000023A02D16550> server_hostname='api.mistral.ai' timeout=120
DEBUG:httpcore.connection:start_tls.complete return_value=<httpcore._backends.sync.SyncStream object at 0x0000023A02ED0950>
DEBUG:httpcore.http11:send_request_headers.started request=<Request [b'POST']>
DEBUG:httpcore.http11:send_request_headers.complete
DEBUG:httpcore.http11:send_request_body.started request=<Request [b'POST']>
DEBUG:httpcore.http11:send_request_body.complete
DEBUG:httpcore.http11:receive_response_headers.started request=<Request [b'POST']>
DEBUG:httpcore.http11:receive_response_headers.complete


Réponse : En réponse à votre question : "Depuis le 27 Février 2025, mon dossier a été mis en paiement. Quand notre association va-t-elle recevoir sa subvention s'il vous plait? Je ne comprends pas trop les deadlines là.", voici les informations demandées :

- **Statut actuel du dossier 82-2415** : Mandatement
- **Date de dernière modification** : 27/02/2025
- **Informations pertinentes du demandeur** :
  - Nom de l'usager : Kassim Riday
  - Montant : 3200 €

En réponse à votre question : "Quand notre association va-t-elle recevoir sa subvention s'il vous plait?" :
- Le statut de votre dossier est actuellement "Mandatement", ce qui signifie que le paiement est en cours de traitement.
- Le paiement devrait être effectué dans les jours suivants.

Pour plus d'informations, vous pouvez contacter l'agent affecté à votre dossier : anaelle.nimba@cr-reunion.fr.

En vous remerciant de votre compréhension.

Cordialement,

[Document: Échanges, Catégorie: Échanges, Section: Dossier 82-2415, Page: 

NameError: name 'python' is not defined